In [2]:
pip install langchain openai faiss-cpu sentence-transformers


Note: you may need to restart the kernel to use updated packages.


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

!pip install langchain langchain-community
!pip install faiss-cpu sentence-transformers numpy



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
#from langchain.llms import OpenAi
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# Load CSV file containing queries
df = pd.read_csv("diabetes_queries.csv")

# Assuming the column containing queries is named 'query'
queries = df["Query"].tolist()

# Load a pre-trained SentenceTransformer model for embedding
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Efficient and fast model

# Generate embeddings for queries
query_embeddings = embedding_model.encode(queries)

# Convert to NumPy array
query_embeddings = np.array(query_embeddings)
# Initialize FAISS index with L2 (Euclidean) distance
dimension = query_embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)

# Add query embeddings to FAISS
faiss_index.add(query_embeddings)

# Save FAISS index for later use
faiss.write_index(faiss_index, "query_index.faiss")

print("FAISS Index successfully created and saved!")



FAISS Index successfully created and saved!


In [7]:
from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_name)  # Force download


In [3]:
import faiss

# Load the FAISS index from the saved file
faiss_index = faiss.read_index("query_index.faiss")

print("FAISS Index successfully loaded!")


FAISS Index successfully loaded!


In [10]:
search_query = "How many carbs does pineapple have"  # Example query

# Convert the query to an embedding
search_embedding = embedding_model.encode([search_query]).astype('float32')

# Search for top 5 most similar queries
D, I = faiss_index.search(search_embedding, 5)  # Returns distances (D) and indices (I)

print("Top similar queries:")
for idx in I[0]:
    print(queries[idx])  # Print the matching queries from your dataset


Top similar queries:
How many carbs are in a bowl of poha?
How many carbs are in a glass of buttermilk?
I get nervous eating at restaurants because I don’t know the carb counts.
How do I estimate carbs in a homemade chapati?
How do I calculate my insulin-to-carb ratio for a high-protein, low-carb meal?


In [9]:
import pandas as pd

# Load queries and their categories
df = pd.read_csv("diabetes_queries.csv")

# Convert queries and categories to lists
queries = df["Query"].tolist()
categories = df["Category"].tolist()
def get_top_category(search_query):
    # Convert the query to an embedding
    search_embedding = embedding_model.encode([search_query]).astype('float32')

    # Search for the most similar query
    D, I = faiss_index.search(search_embedding, 1)  # Top 1 match

    # Get the best-matching query's index
    best_match_index = I[0][0]

    # Retrieve the category of the matched query
    top_category = categories[best_match_index]

    return top_category

# Example usage
query = "How many carbs does pineapple"
category = get_top_category(query)
print(f"Predicted Category: {category}")



Predicted Category: food/nutrition
